# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i f ilein] [-o f ileout]


In [14]:
import gmpy2
from gmpy2 import mpz
import sympy
import numpy as np

In [15]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides(a,b):\n    if a%b == 0:\n        return b\n    else:\n        return algoritmo_euclides(b, a%b)\n'

In [16]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [17]:
mcd=algoritmo_euclides(39,150)
print(mcd)

3


In [18]:
algoritmo_euclides(7,15)

mpz(1)

In [19]:
"""
VERSIÓN SIN GMP 

def algoritmo_euclides_extendido(a,b):

    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v
"""

'\nVERSIÓN SIN GMP \n\ndef algoritmo_euclides_extendido(a,b):\n\n    # Identidad de Bézout 1=u*a + v*b\n    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v\n    if a==0:\n        mcd=b\n        u=0\n        v=1\n    else:\n        mcd, x, y = algoritmo_euclides_extendido(b%a, a)\n        u=y-(b//a)*x\n        v=x\n        \n    return mcd, u, v\n'

In [20]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [21]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        return inv
    else:
        print("Error")

In [646]:
inverso(51,23)

mpz(-9)

In [22]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
        file.close()
    print("Cadena: {}".format(cadena))
    return cadena

In [23]:
def print_output(o,cadena):
    if o==0:
        print("Cadena: {}".format(cadena))
    else:
        file=open(o, "w")
        cadenaToStr = ' '.join([str(elem) for elem in cadena])
        file.write(cadenaToStr)
        file.close()

In [693]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            print(cadena_cifrada)
            resul=""
            for i in range(len(cadena_cifrada)):
                   resul=resul+alfabeto[cadena_cifrada[i]]
                    
            print_output(o,resul)
            
        elif modo=="-D":
            
            cadena_cifrada=read_input(i)
            #cadena_cifrada=cadena_cifrada.split(", ")
            
            cadena_numerica=[]
            for k in cadena_cifrada:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            cadena_descifrada=[]
            cadena_texto=""
            #inv=inverso(m,a)
            inv=pow(a, -1, m)
            print(inv)
            for i in range(len(cadena_numerica)):
                cadena_numerica[i]=int(cadena_numerica[i])
                
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto=cadena_texto+alfabeto[k_descifrado]
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [694]:
afin("-C",51,23,3,"cadena.txt")

Cadena: Hola Nueva York!
[48, 19, 1, 3, 33, 4, 44, 27, 3, 31, 19, 37, 29]
Cadena: WtbdHeSBdFtLD


In [695]:
afin("-C",130,16,27)

16 y 130 no son primos relativos. Error


In [943]:
afin("-C",51,13,0)

antiaereo
Cadena: antiaereo
[0, 16, 43, 2, 0, 1, 17, 1, 29]
Cadena: aqRcabrbD


In [820]:
afin("-D",52,33,38)

jGLMaGsGoiPMioinoBGUwoPGHGtQoZ
Cadena: jGLMaGsGoiPMioinoBGUwoPGHGtQoZ
41
Cadena: holacomoestasesperoquetodobien


In [343]:
afin("-D",52,23,3,"cadena_cifrada.txt","resultado.txt")

Cadena: 34, 13, 48, 3, 16, 47, 43, 18, 3, 9, 13, 30, 25


## 1.b Criptoanálisis del cifrado afín

In [850]:
def fortaleza(m):
    z_m_inv=sympy.totient(m) #calculamos la funcion phi
    return gmpy2.mul((m),z_m_inv) #calculamos la fortaleza multiplicando Zm y Zm*


In [851]:
print(fortaleza(26**26+26))
#fortaleza(26) este seria el resultado si no aceptaramos mayúsculas
print(fortaleza(52))


11545444563871328761349212098135488565445348609393477048015277366400000000
1248


In [344]:
alfabeto='abcdefghijklmnopqrstuvwxyz'
digrama=([])
for i in alfabeto:
    for j in alfabeto:
        digrama.append(i+j)



In [862]:
def afin_no_trivial(modo, m, a, b, i=0, o=0):
    
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    digrama=([]) #generamos un vetor de di-gramas del alfabeto que hemos declarado arriba
    for i in alfabeto:
        for j in alfabeto:
            digrama.append(i+j)
    
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            #if i==0:
            cadena=input()
            #cadena=read_input(i)
            #else:
            #    file=open(i, "r")
             #   cadena=file.read()
              #  file.close()
            
            #Traducimos los caracteres a números para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena)%2==0:
                while j <(len(cadena)-1):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
            else:
                while j <(len(cadena)-2):
                    cadena_numerica.append(digrama.index(cadena[j]+cadena[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena[len(cadena)-1])) #como es impar, la última letra la ciframos aparte
            
            #utilizamos la función de cifrado
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            #pasamos el resultado a caracteres y lo guardamos como string
            resul=""
            for k in cadena_cifrada:
                resul=resul+digrama[k]
          
            #si no se ha pasado un fichero de salida por parámetro, imprimimos el resultado
            print_output(o,resul)
            
        elif modo=="-D":
            
            #cadena_cifrada=read_input(i)
            cadena_cifrada=input()
    
            cadena_descifrada=[]
            cadena_texto=""
            
             #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            #if i==0:
             #    cadena_cifrada=input()
            #cadena=read_input(i)
            #else:
             #   file=open(i, "r")
             #   cadena_cifrada=file.read()
              #  file.close()
                
            #obtenemos el inverso en el modulo para poder utilizar la función de descifrado
            #ya hemos comprobado al principio que el inverso existe.
           
            #inv=inverso(a,m)
            inv=pow(a, -1, m)
            
            #pasamos el texto a un formato numérico para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena_cifrada)%2==0:
                while j <(len(cadena_cifrada)-1):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
            else:
                while j <(len(cadena_cifrada)-2):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena_cifrada[len(cadena_cifrada)-1])) #como es impar, la última letra la ciframos aparte
            
            #desciframos el texto con la función de descifrado: 
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0: #ajustamos el modulo
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
            
            #pasamos el texto a caracteres
            if len(cadena_cifrada)%2==0:
                for k in cadena_descifrada:
                    cadena_texto=cadena_texto+digrama[k]
            else:       
                for k in range(len(cadena_descifrada)-1):
                    cadena_texto=cadena_texto+digrama[cadena_descifrada[k]]
                    
                cadena_texto=cadena_texto+alfabeto[cadena_descifrada[(len(cadena_descifrada)-1)]]
            
            #si no se ha pasado un archivo para guardar el resultado, se imprime por pantalla
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [879]:
afin_no_trivial("-C",701,23,3)

esternocleidomastoideo
Cadena: afjhzeajoazajfqbsdzaxq


In [859]:
afin_no_trivial("-D",701,52,3)

abcd


FileNotFoundError: [Errno 2] No such file or directory: 'z'

In [986]:
from collections import Counter

cadena = "aqRcabrbD"
letters = Counter(cadena)

castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])

ingles=(['a', 11.96],['b', 1.54],['c', 3.06],['d', 3.99],['e', 12.51],['f', 2.30],['g', 1.96],
        ['h', 0.89],['i', 7.26],['j', 0.16],['k', 0.67],['l', 4.14],['m', 2.53],['n', 7.09],
       ['o', 7.60],['p', 2.0],['q', 0.11],['r', 6.12],['s', 6,54],['t', 9.25],['u', 2.71],
       ['v', 0.99],['w', 1.92],['x', 1.92],['y', 1.73],['z', 0.19])

letras=sorted(letters, reverse=True)
castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
ingles_ord=sorted(ingles, key=lambda letra: letra[1], reverse=True)


print("segunda hipotesis")
c1=alfabeto.index("b")
c2=alfabeto.index("a")
t1=alfabeto.index(castellano_ord[0][0])
t2=alfabeto.index(castellano_ord[1][0])

#pos(c1)=pos(t1)*a+b -> 1=4*a+b->a=1*inv(4)
#-
#pos(c2)=pos(t1)*a+b -> 0=0*a+b ->b=0

#pos(c1)-pos(c2)=(pos(t1)-pos(t2))*a

#1=(-4)*
#0-1=(4-0)*a

#a=1*inv(4)

#comprobamos que hemos resuelto bien la ecuación
a=1*pow(4,-1,51)
#comprobamos que sean co-primos
if algoritmo_euclides(a,51)==1:
        b=c1-int(a)*t1
        print("es correcto, antiaereo se cifro con 13 y 0")
        #print(afin("-D", 51, a, b))
        
              
    

segunda hipotesis
es correcto, antiaereo se cifro con 13 y 0


## 2. Sustitución polialfabeto

## 2.a Método de Hill

In [47]:
import numpy as np
import os
import math
import copy

In [ ]:
Funcion que cálcula el determinante de una matriz

In [48]:
def determinante(matriz):
   
    if len(matriz)==2 and len(matriz[0])==2:
        #calculamos el determinante
        det=matriz[0][0]*matriz[1][1]-(matriz[1][0]*matriz[0][1])
       
        return det
    else:
        suma=0
        for i in range(len(matriz)): #calculamos el determinante por cofactores
            maux=copy.deepcopy(matriz)
            maux.remove(matriz[0]) #eliminamos la primera fila
            for j in range(len(maux)):
                maux[j]=maux[j][0:i]+maux[j][i+1:]
                
         
            suma= suma+ (-1)**((i+j)%2)*matriz[0][i]*determinante(maux)
            
        return suma
        

In [49]:
#comprobación de la función
matriz = [[1,2,3], [3,4,5], [1,4,3]]
print(determinante(matriz))
matriz = [[11,8], [3,7]]
print(determinante(matriz))

-8
53


In [ ]:
Función que cálcula el adjunto de una matriz

In [50]:
def adjunto(matriz):
    adjunto=np.zeros(np.shape(matriz))
    if len(matriz)==2 and len(matriz[0])==2:
         #calculamos el adjunto
        adjunto[0][0]=matriz[1][1]
        adjunto[0][1]=-matriz[0][1]
        adjunto[1][0]=-matriz[1][0]
        adjunto[1][1]=matriz[0][0]
        
        return adjunto
    else:
        
        for i in range(len(matriz)):
            maux=copy.deepcopy(matriz)
            for j in range(len(matriz)):
             
                maux=np.delete(matriz,i,0)
                aux=np.delete(maux,j,1)
                auxi=aux.tolist()
                #la matriz de cofactores transpuesta es el djunto
                adjunto[j][i]=(-1)**((i+j)%2)*determinante(auxi)
            
                
        return adjunto

In [51]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(adjunto(matriz))

[[ 7. -8.]
 [-3. 11.]]


In [ ]:
Función que cálcula la inversa de una matriz

In [52]:
def inversa(matriz,modulo):
    inversa=np.zeros(np.shape(matriz))
    det=determinante(matriz)%modulo
    if det !=0:
        adj=adjunto(matriz)%modulo
        for i in range(len(matriz)):
            for j in range(len(matriz[i])):
                inversa[i][j]=(adj[i][j]/det)#%modulo
                
    return inversa%modulo #esto puede que no sea necesario porque ya estamos en matemática modular

In [53]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(inversa(matriz,26))

[[ 7. 18.]
 [23. 11.]]


In [54]:
arr = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
np.delete(arr, 1, 0)

array([[ 1,  2,  3,  4],
       [ 9, 10, 11, 12]])

In [55]:
matriz = [[1, 2, 4], [3,5,4], [1]]

# Padding a la matriz
n=3
print(matriz)
print(len(matriz[n-1]))
k=len(matriz[n-1])
if k<n:
    padding=[0]*(n)
    for i in range(0,k):
        padding[i]=matriz[n-1][i]
    matriz[n-1]=padding
  
print(matriz)
matriz=np.array(matriz)
matriz%3

[[1, 2, 4], [3, 5, 4], [1]]
1
[[1, 2, 4], [3, 5, 4], [1, 0, 0]]


array([[1, 2, 1],
       [0, 2, 1],
       [1, 0, 0]], dtype=int32)

In [56]:
print(matriz[1])

[3 5 4]


In [ ]:
Función de cifrado del algoritmo.
Parámetros:
    -matriz_numerica: el texto a cifrar en formato matriz de numeros
    -matriz: matriz de transformación
    -mod: modulo en el que trabajamos
    -n: dimensión 

In [57]:
def cifrar(matriz_numerica, matriz,mod,n):
    matriz_cifrada=np.zeros((n,n))
    
    for i in range(len(matriz_numerica)):
        cadena_cifrada= (np.dot(matriz_numerica[i],matriz))%mod
        matriz_cifrada[i]=cadena_cifrada

    return matriz_cifrada  

In [ ]:
Función de descifrado del algoritmo.
Parámetros:
    -matriz_cifrada: el cifrado a descifrar en formato matriz de numeros
    -matriz: matriz de transformación
    -mod: modulo en el que trabajamos
    -n: dimensión

In [58]:
def descifrar(matriz_cifrada, matriz,mod,n):
    matriz_descifrada=np.zeros((n,n))
    inv=inversa(matriz,mod)
 
    for i in range(len(matriz_cifrada)):
        cadena_descifrada= (np.dot(matriz_cifrada[i],inv))%mod
        matriz_descifrada[i]=cadena_descifrada

    return matriz_descifrada  

In [59]:
def hill(modo,mod,n,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
  
    #leemos la matriz de transformación del archivo y la guardamos
    with open(k,'r') as f:
        datos = ''.join(f.readlines()).replace('\n',';')
    matriz = np.matrix(datos).tolist()
    f.close()
   
    #cálculamos el determinante de la matriz
    det=np.linalg.det(matriz)
    #comprobamos que la matriz K tiene una función biyectiva
    if algoritmo_euclides(int(det),mod)==1:
       
        if modo=="-C":
           
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(cadena_numerica)/n
            max=len(cadena_numerica)
            matriz_numerica=np.zeros((math.ceil(m),n))
         
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<max:
                        matriz_numerica[i][j]=cadena_numerica[pos]
                        pos=pos+1
                        
            #ciframos cadena a cadena y lo guardamos en un matriz           
            matriz_cifrada=cifrar(matriz_numerica,matriz,mod,n)
            print_output(o,matriz_cifrada)
           
            
        elif modo=="-D":
            if i==0:
                cadena_cifrada=input()
                mat_cifrada = np.matrix(cadena_cifrada).tolist()
                matriz_cifrada= np.reshape(mat_cifrada, (n,n))
               
                        
            else:
                
                with open(i,'r') as f:
                    datos = ''.join(f.readlines()).replace('\n',';')     
                f.close()
                
                matriz_cifrada = np.matrix(datos).tolist() 
        
           
            matriz_descifrada=descifrar(matriz_cifrada, matriz,mod,n)
            
            cadena_num=[]*n*n
            for i in range(len(matriz_descifrada)):
                for j in range(len(matriz_descifrada[i])):
                    
                    if matriz_descifrada[i][j]<0:
                        matriz_descifrada[i][j]=mod+matriz_descifrada[i][j]
        
                    cadena_num.append(alfabeto[int(matriz_descifrada[i][j])])
            print_output(o,cadena_num)
            
    else:
        print("{} y {} no son primos relativos. Error".format(det,mod))

In [60]:
k = [[11,8], [3,7]]
hill("-D",26, 2,"matriz_k.txt","matriz_cifrada.txt" ,0)

Cadena: ['h', 'o', 'l', 'a']


In [61]:
hill("-C",26,2, "matriz_k.txt", 0,0 )


Cadena: 
Cadena: [[0. 0.]
 [0. 0.]]


In [62]:
cadena_numerica=[8,15,12,1]
matriz = [[11,8], [3,7]]
n=2

m=len(cadena_numerica)/n
max=len(cadena_numerica)
matriz_numerica=np.zeros((math.ceil(m),n))

pos=0
for i in range(math.ceil(m)):

    for j in range(n):
        
        if pos<max:
           
            matriz_numerica[i][j]=cadena_numerica[pos]
            pos=pos+1
            
print("matriz_numerica")
print(matriz_numerica)   

matriz_cifrada=cifrar(matriz_numerica, matriz,26,n)
print("matriz cifrada")
print(cifrar(matriz_numerica, matriz,26,n))

print(descifrar(matriz_cifrada, matriz,26,n))
print("matriz descifrada")
matriz_descifrada=descifrar(matriz_cifrada, matriz,26,n)
alfabeto='abcdefghijklmnopqrstuvwxyz'
cadena_num=[]*n*n
for i in range(len(matriz_descifrada)):
    for j in range(len(matriz_descifrada[i])):
        if matriz_descifrada[i][j]<0:
            matriz_descifrada[i][j]=26+matriz_descifrada[i][j]
        cadena_num.append(alfabeto[int(matriz_descifrada[i][j])-1])
print(cadena_num)

matriz_numerica
[[ 8. 15.]
 [12.  1.]]
matriz cifrada
[[ 3. 13.]
 [ 5. 25.]]
[[ 8. 15.]
 [12.  1.]]
matriz descifrada
['h', 'o', 'l', 'a']


In [63]:
with open('matriz_k.txt','r') as f:
    datos = ''.join(f.readlines()).replace('\n',';')

matriz = np.matrix(datos)
print(matriz)

[[11  8]
 [ 3  7]]


## 2.b Método de Vigenere

In [ ]:
El siguiente programa implementa el método. de Vigenere.

Llamada a la función: 
vigenere {-C|-D} {-k clave} [-i f ilein] [-o f ileout]

-k cadena de caracteres usada como clave

In [476]:
def vigenere(modo,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    base=len(alfabeto)
    n=len(k)
    #Traducimos la clave de caracteres a números
    k_numerica=[]
    for j in k:
        if j in alfabeto: 
            k_numerica.append(alfabeto.index(j))
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el input
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_numerica)/n
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que cifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_numerica.shape[0]
        elementos=matriz_numerica.shape[1]
        #print("Matriz numerica")
        #print(matriz_numerica)
        matriz_cifrada=np.zeros((filas,elementos))
        #print("Matriz cifrada de ceros")
        #print(matriz_cifrada)
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_cifrada[i][j]=(matriz_numerica[i][j]+k_numerica[j])%base
        #print("Matriz cifrada")
        #print(matriz_cifrada)
        cadena_cifrada=np.concatenate(matriz_cifrada)
        resul=""
        for i in cadena_cifrada:
            resul=resul+alfabeto[int(i)]
        #print(cadena_cifrada)
        print_output(o,resul)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        #for i in range(len(cadena_cifrada)):
        #   cadena_cifrada[i]=int(cadena_cifrada[i])
        # Dividimos en bloques de n elementos el texto cifrado
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_cifrada)/n
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que descifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_cifrada.shape[0]
        elementos=matriz_cifrada.shape[1]
        matriz_descifrada=np.zeros((filas,elementos))
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_descifrada[i][j]=(matriz_cifrada[i][j]-k_numerica[j])%base
        cadena_descifrada=np.concatenate(matriz_descifrada)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [849]:
vigenere("-C", "clave")

KeyboardInterrupt: Interrupted by user

In [579]:
vigenere("-D", "clave")


48.0 24.0 8.0 42.0 8.0 19.0 29.0 8.0 24.0 4.0 5.0 37.0 20.0 40.0 18.0 15.0 25.0 12.0 21.0 7.0 6.0 49.0 0.0 24.0 21.0 10.0 14.0 0.0 21.0 4.0
Cadena: 48.0 24.0 8.0 42.0 8.0 19.0 29.0 8.0 24.0 4.0 5.0 37.0 20.0 40.0 18.0 15.0 25.0 12.0 21.0 7.0 6.0 49.0 0.0 24.0 21.0 10.0 14.0 0.0 21.0 4.0
Bloques ok
Cadena: ['U', 'n', 'i', 'v', 'e', 'r', 's', 'i', 'd', 'a', 'd', 'A', 'u', 't', 'o', 'n', 'o', 'm', 'a', 'd', 'e', 'M', 'a', 'd', 'r', 'i', 'd', 'a', 'a', 'a']


In [66]:
vigenere("-C", "clave", "texto_vigenere.txt", "resultado_vigenere.txt")

Cadena: Universidad Autonoma de Madrid
Bloques ok


In [489]:
vigenere("-D", "clave", "resultado_vigenere.txt", "descifrado_vigenere.txt")

Cadena: 48.0 24.0 8.0 42.0 8.0 19.0 29.0 8.0 24.0 4.0 5.0 37.0 20.0 40.0 18.0 15.0 25.0 12.0 21.0 7.0 6.0 49.0 0.0 24.0 21.0 10.0 14.0 0.0 21.0 4.0
Bloques ok


## 2.c Criptoanálisis del cifrado de Vigenere

El siguiente programa implementa el indice de coincidencia.

Llamada a la función: 
IC {-l Ngrama} [-i f ilein] [-o f ileout]
-l longitud de n-grama buscado


In [ ]:
El siguiente programa implementa el test de kasiski.

Llamada a la función: 
kasiski {-l Ngrama} [-i f ilein] [-o f ileout]
-l longitud de n-grama buscado

In [68]:
def calcular_divisores(n):
    lista = []
    for i in range(2,n):
        if n % i == 0:
            lista.append(i)
    return lista

In [480]:
n=25
print(calcular_divisores(n))

[5]


In [501]:
def obtener_tuplas(lista):
    res = {}
    freq =[]
    cont = 0
    i = 0
    while i < len(lista): 
        elt= lista[i:i+3] # Cogemos al menos 3 caracteres como tamaño de la tupla
        tam = len(elt)
        if tam == 3: #tiene que ser 3, si no estamos al final de la lista
            for j in range(i+1,len(lista)): #Find further in the list for the same pattern
                if lista[i:i+tam] == lista[j:j+tam]: #Si coinciden, seguimos comprobando
                    while lista[i:i+tam] == lista[j:j+tam]:
                        tam = tam + 1
                    tam = tam -1
                    elt = lista[i:i+tam] #Ahora tenemos una tupla 
                    dist = j - i #calculamos la distancia
                    freq.extend(calcular_divisores(dist)) #Añadimos los divisores a la lista 
                    print ("%s\ti:%s\tj:%s\tdiff:%s\t\tDivisors:%s" % (elt,i,j, dist,calcular_divisores(dist))) #Print information about the tuple (can be deleted)
                    cont = cont +1
                    j = j + tam + 1
            i = i + tam -3 +1
        else:
            i = i + 1
    return cont, freq

In [502]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"
cont, freq=obtener_tuplas(lista)
print(cont)
print(freq)

uwpt	i:4	j:29	diff:25		Divisors:[5]
waH	i:11	j:46	diff:35		Divisors:[5, 7]
2
[5, 5, 7]


In [554]:
def contDiv(lista): # devuelve una lista con (caracter_decimal, occ) 
    d={}
    for elt in lista:
        if d.__contains__(elt): #contamos los divisores de cada uno
            d[elt] += 1
        else:
            d[elt] = 1
    return sorted(d.items(),key=lambda x: x[1], reverse=True) #ordenamos la lista en orden descendente

In [590]:
def dividir_lista(key,lista): #ahora que sabemos la cardinalidad de la llave, dividimos la lista en ese modulo
    dic = {}
    for elem in range(key): #hacemos sublistas de key
        dic[elem] = []
        
    i = 0
    for j in range (len(lista)):
        if i == key: #si el inidice es igual a key hemos llegado al final de la sublista
            i = 0 #para la proxima sublista
        dic[i].append(lista[j])
        i = i + 1
    return dic

In [607]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"

key=5
#dividir_lista(key, lista)

In [592]:
def descifrar_v(lista,dist):
    alfabeto="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lista1=list()
    for elem in lista:
        valor = alfabeto.index(elem) - int(dist)
        if valor < 0:
            lista1.append(256 + (valor % -256))
        else:
            lista1.append(valor)
    return lista1

In [593]:
lista=input()

cont,freq=obtener_tuplas(lista)
l=contOcc(freq)
res = dividir_lista(l[i][0], lista) 
res=contOcc(freq)
occ = contOcc(res[i]) 
print(res[1])
print(occ[0][0])
print(lista[0])
print(l)
descifrar_v(res[0],occ[0][0])

TypeError: list indices must be integers or slices, not str

In [594]:
def recrear_lista(dic):
    i = 0
    output = []
    try:
        while 1:
            for l in dic.values():
                output.append(l[i])
            i = i + 1
    except:
        pass
    return output

In [722]:
def criptoanalisis_vigenere(lista):
        alfabeto="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        #cadena=input() #obtenemos la cadena a descifrar
        #lista=[]
        #for elem in cadena:
            #lista.append(alfabeto.index(elem)) #pasamos las letras a numeros
        
        #buscamos las tuplas del texto y la distancia entre ellas
        cont,freq=obtener_tuplas(lista)
        lista_modulos_posibles=contDiv(freq) #devuelve un lista con (divisor, #divisor)
        #la lista_modulos_posibles esta ordenada del que más aparece al que menos, asique elegimos el primer elemento
        
        res = dividir_lista(lista_modulos_posibles[0][0], lista) #We consider in this exemple a key length of 10 and l the original list
        #tenemos la lista original dividida en lista_modulos_posibles[0][0] elementos
        print(len(res))
        for i in range(lista_modulos_posibles[0][0]): #por cada sublista
            
            divisores = contDiv(res[i]) # Hacemos un analisis de frecuecia
            shift = (alfabeto.index(divisores[0][0]) - 32) % 256 # Consider the most frequent element of being a space(32 in decimal)
           
            print ("Frequency analysis for the index: %s\tshift:%s\n%s\n" % (i,shift,occ)) #Print informations (can be deleted)
            res[i] = descifrar_v(res[i],shift) #Intnemos descifrarlo 

        final = recrear_lista(res) #Once we have processed all sub-list recreate a list with all the sub-lists.
        print(final)
        print(len(alfabeto))
        resul=""
        for elem in final:
            resul=resul+alfabeto[elem-1]
        print(resul)
        #print (''.join([chr(x) for x in final])) #Print the result

In [846]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"
lista1="jzlviuEawerpnNepoozroFcCeunoNeuDowvgEoysgycJwcDqPipztDippmPgjzszrvtdJgqxozwvpeEitnixmqEoOenxeIxgtnOitxiIedweLygyoxspDiBsfpsxmhCaMtgCoLygllHipzsxsoAiGesFevpizeNenrove"

cont, freq=obtener_tuplas(lista)
print(cont)
print(freq)
criptoanalisis_vigenere(lista1)

uwpt	i:4	j:29	diff:25		Divisors:[5]
waH	i:11	j:46	diff:35		Divisors:[5, 7]
2
[5, 5, 7]
ipz	i:49	j:139	diff:90		Divisors:[2, 3, 5, 6, 9, 10, 15, 18, 30, 45]
Lyg	i:108	j:133	diff:25		Divisors:[5]
5
Frequency analysis for the index: 0	shift:230
[(3, 1), (49, 1)]

Frequency analysis for the index: 1	shift:249
[(3, 1), (49, 1)]

Frequency analysis for the index: 2	shift:238
[(3, 1), (49, 1)]

Frequency analysis for the index: 3	shift:247
[(3, 1), (49, 1)]

Frequency analysis for the index: 4	shift:228
[(3, 1), (49, 1)]

[35, 32, 29, 30, 36, 46, 37, 18, 31, 32, 43, 22, 31, 48, 32, 41, 21, 32, 34, 45, 40, 38, 20, 37, 32, 46, 20, 32, 48, 32, 46, 36, 32, 31, 49, 32, 37, 32, 33, 46, 32, 31, 20, 44, 50, 28, 36, 34, 50, 36, 41, 32, 37, 38, 36, 41, 22, 30, 50, 34, 35, 32, 36, 34, 45, 47, 26, 21, 44, 34, 42, 30, 32, 34, 50, 47, 22, 22, 39, 36, 45, 20, 26, 32, 40, 42, 37, 32, 49, 32, 39, 30, 22, 43, 51, 32, 26, 31, 49, 36, 45, 30, 26, 43, 32, 29, 29, 22, 46, 52, 32, 31, 32, 32, 46, 41, 36, 26, 36, 46

## 3. Cifrado de flujo

In [ ]:
def rec_fib(n):
    if n > 1:
        return rec_fib(n-1) + rec_fib(n-2)
    return n

In [ ]:
# Generador de secuencia aleatoria
def generador_aleatorio(m,cont):
    k=(rec_fib(m)%m)*m*cont
    return k

In [ ]:
# Ejemplo de secuencia cifrante de 5 elementos para clave 14
m=14
for i in range(5):
    k=generador_aleatorio(m,i)
    print(k)

In [ ]:
# m es la clave
# n es el tamaño de la secuencia de claves
def flujo(modo,m,n,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Ciframos carácter a carácter
        cadena_cifrada=[]
        count=0
        for i in cadena_numerica:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_cifrada.append(int(bin(i^k)[2:]))
            count=count+1
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        cadena_descifrada=[]
        cadena_texto=[]
        for i in range(len(cadena_cifrada)):
            cadena_cifrada[i]=int(cadena_cifrada[i],2)
        count=0
        for i in cadena_cifrada:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_descifrada.append((i^k))
            count=count+1
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [ ]:
flujo("-C", 4,2)

In [ ]:
flujo("-D", 4,2)

In [ ]:
flujo("-C", 10,20)

In [ ]:
flujo("-D", 10, 20)